In [0]:
from pyspark.sql.functions import col, lit, concat_ws, collect_list
spark.sql("USE CATALOG anime_warehouse")

DataFrame[]

In [0]:
print("dim_anime_gold")

# Join silver anime with a collapsed version of genres for easy filtering
silver_anime = spark.read.table("anime_warehouse.silver.dim_anime")
silver_genres = spark.read.table("anime_warehouse.silver.dim_genres")

# Group genres so each anime has one row with a comma-separated list of genres
import pyspark.sql.functions as F
genre_list = silver_genres.groupBy("anime_id").agg(F.concat_ws(", ", F.collect_list("genre")).alias("all_genres"))

gold_anime = silver_anime.join(genre_list, on="anime_id", how="left")

gold_anime.write.format("delta").mode("overwrite").saveAsTable("anime_warehouse.gold.dim_anime_gold")

dim_anime_gold


* Combining the core anime data with its primary genres so that a user doesn't have to join two tables just to see what kind of show it is
* Dim genre (could do what the previous code cell did, but it's hard to analyze by genre (have to use LIKE statement to sort by genre))
    * Could snowflake it and have a bridge table that connects to anime_dim for a manytomany (genre to anime) relationship --> many to many relationship in a star schema makes things a bit difficult in performance and design but that's what bridge tables allow for in the kimball model
    * Want to answer the question of e.g. "How do comedy anime compare to drama?"

In [0]:
print("fact_anime_performance")

# Extract only the keys and the quantitative metrics
fact_performance = spark.read.table("anime_warehouse.silver.dim_anime").select(
    col("anime_id"),
    col("score"),
    col("rank"),
    col("popularity"),
    col("members"),
    col("episodes")
)

fact_performance.write.format("delta").mode("overwrite").saveAsTable("anime_warehouse.gold.fact_anime_performance")

fact_anime_performance


* Numbers table for rankings and scores

In [0]:
print("studio_performance_denormalized")

dim_anime = spark.read.table("anime_warehouse.gold.dim_anime_gold")
bridge_companies = spark.read.table("anime_warehouse.silver.bridge_anime_companies")
dim_entities = spark.read.table("anime_warehouse.silver.dim_entities")

# Filter for only studios
studios = dim_entities.filter(col("entity_type") == "studio")

# Join them all together
studio_gold = (dim_anime
    .join(bridge_companies, "anime_id")
    .join(studios, bridge_companies.company_id == studios.entity_id)
    .select(
        dim_anime.anime_id,
        dim_anime.title,
        dim_anime.score,
        studios.name.alias("studio_name"),
        dim_anime.all_genres,
        dim_anime.type,
        dim_anime.start_date
    )
)

studio_gold.write.format("delta").mode("overwrite").saveAsTable("anime_warehouse.gold.studio_performance_denormalized")

studio_performance_denormalized


* specialized gold table that answers "which studios are responsible for which high-rated shows?"

In [0]:
%sql
SELECT 
  studio_name, 
  ROUND(AVG(score), 2) as average_rating, 
  COUNT(*) as total_titles
FROM anime_warehouse.gold.studio_performance_denormalized
WHERE score IS NOT NULL
GROUP BY studio_name
HAVING total_titles > 5
ORDER BY average_rating DESC
LIMIT 50;

studio_name,average_rating,total_titles
Studio Signpost,8.14,7
Sharefun Studio,8.09,6
Shenman Entertainment,8.05,7
Studio Bind,7.96,9
Shuka,7.82,20
Motion Magic,7.71,16
Original Force,7.68,11
Animation Do,7.63,10
Red Dog Culture House,7.6,11
CygamesPictures,7.58,16


In [0]:
%sql
SELECT